In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-05-21"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
11214,2024-05-21,Macedônia Do Norte Prva Liga,14:00,TFT Skopje,Rabotnicki,1.83,1.83,156.5,1.83,1.83,0.0,0.00,0.00,Sr8nNXuC,0.546448,0.546448,0.546448,0.546448,0.092896,273.264,133.803665,0.489650,0.6,1.341641,2.236068,391.46,3.882,1.659765,0.427554,-50.0,137.910,51.282521,0.371855,1.2,1.643168,1.369306,96.05,1.796,0.598732,0.333370,-20.0,74,85,5.29,1.13,135.230,156.624,0.000000,0.000000,NaN,0.21,0.042,19.761905,0.000000,0.0,0.000000,-1.19,-0.238,-3.487395,0.634865,0.8,0.165135
11215,2024-05-21,Macedônia Do Norte Prva Liga,14:00,MZT Skopje,Pelister,1.37,2.91,154.5,1.85,1.85,0.0,0.00,0.00,ClaURNWG,0.729927,0.343643,0.540541,0.540541,0.073570,105.108,12.412354,0.118091,3.0,0.000000,0.000000,122.18,1.258,0.153688,0.122169,71.0,169.676,45.411766,0.267638,1.8,1.643168,0.912871,192.50,2.110,0.565685,0.268097,26.0,82,77,1.49,2.50,100.876,165.732,0.508853,0.000000,NaN,0.91,0.182,2.032967,0.844003,0.8,-0.044003,0.72,0.144,13.263889,0.499648,0.6,0.100352
11216,2024-05-21,Lituânia Lkl,12:50,Zalgiris Kaunas,Lietkabelis,1.12,5.95,161.5,1.87,1.87,0.0,0.00,0.00,ALGg8AZj,0.892857,0.168067,0.534759,0.534759,0.060924,110.676,15.925454,0.143893,1.8,1.643168,0.912871,97.52,1.120,0.129615,0.115728,75.0,144.126,40.790327,0.283019,2.4,1.341641,0.559017,201.00,1.594,0.410219,0.257352,21.0,92,100,1.06,2.01,102.664,192.478,0.966146,0.000000,NaN,0.38,0.076,1.578947,0.903270,1.0,0.096730,0.59,0.118,41.949153,0.571962,0.6,0.028038
11217,2024-05-21,Itália Série A2,15:30,Forli,Trieste,1.57,2.31,152.5,1.86,1.80,0.0,0.00,0.00,r9u5UVNi,0.636943,0.432900,0.537634,0.555556,0.069843,104.272,16.725553,0.160403,1.8,1.643168,0.912871,133.00,1.312,0.157861,0.120321,38.0,177.544,75.522776,0.425375,1.8,1.643168,0.912871,101.38,2.240,1.065622,0.475724,3.0,95,74,1.40,1.37,99.888,206.644,0.269721,0.023184,NaN,-1.47,-0.294,-1.938776,0.789999,0.8,0.010001,-0.52,-0.104,-12.596154,0.429718,0.2,-0.229718
11218,2024-05-21,Itália Série A1 Feminina,16:00,Schio F,Venezia F,1.44,2.65,140.5,1.81,1.85,0.0,0.00,0.00,j17YEsAt,0.694444,0.377358,0.552486,0.540541,0.071803,95.876,22.742069,0.237203,2.4,1.341641,0.559017,135.68,1.286,0.470404,0.365789,43.0,103.378,25.343417,0.245153,3.0,0.000000,0.000000,141.95,1.276,0.232551,0.182250,97.0,64,85,2.12,1.67,92.280,86.890,0.418386,0.015456,NaN,1.29,0.258,1.705426,0.000000,0.0,0.000000,-0.02,-0.004,-412.500000,0.829529,0.8,-0.029529
11219,2024-05-21,Itália Série A2,15:30,Fortitudo Bologna,Sebastiani Rieti,1.34,3.08,145.5,1.87,1.83,0.0,0.00,0.00,6mQUORiT,0.746269,0.324675,0.534759,0.546448,0.070944,105.092,6.672535,0.063492,2.4,1.341641,0.559017,97.92,1.376,0.112383,0.081674,59.0,155.320,63.963234,0.411816,3.0,0.000000,0.000000,161.70,1.928,0.638490,0.331167,50.0,64,77,1.53,2.10,105.828,184.348,0.556727,0.015289,NaN,0.30,0.060,5.666667,0.721625,0.8,0.078375,6.75,1.350,1.540741,0.472761,0.8,0.327239
11220,2024-05-21,Polônia Liga De Basquete,12:30,Trefl Sopot,Slask Wroclaw,1.31,3.32,158.5,1.86,1.88,0.0,0.00,0.00,CEb4HwT9,0.763359,0.301205,0.537634,0.531915,0.064564,152.994,33.010631,0.215764,1.8,1.643168,0.912871,203.01,1.674,0.298044,0.178043,14.0,214.020,62.953626,0.294148,1.8,1.643168,0.912871,287.18,2.672,0.828716,0.310148,-11.0,101,83,2.01,3.46,115.468,214.460,0.613946,0.007563,NaN,-1.09,-0.218,-1.422018,0.749300,0.7,-0.049300,3.55,0.7

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
11222,07:05,Japão B.League,Ryukyu,Chiba,2.03,Back Home
11249,21:00,Argentina Liga A,San Martin,Boca Juniors,1.91,Back Home
11250,21:30,Argentina Liga A,La Union,Obras,1.76,Back Home
